[Documentação](https://python.langchain.com/docs/how_to/#text-splitters)
#### Conceito

Divisores de Texto/documentos (Text splitters) é  uma etapa importante de pré-processamento para muitas aplicações. A técnica envolve a divisão de textos grandes em pedaços menores e gerenciáveis. Esse processo oferece vários benefícios, como garantir o processamento consistente de tamanhos variados de documentos, superar as limitações de tamanho de entrada dos modelos e melhorar a qualidade das representações de texto usadas em sistemas de recuperação. Existem várias estratégias para dividir documentos, cada uma com suas próprias vantagens.

O que os splitters esperam como entrada? Eles esperam:
- Uma lista de documentos `List[Documents]` caso você vá usar a função: `split_documents`.
- Um texto caso você vá usar a função `split_text`.

Quais parâmetros conhecer?
- `chunk_size` - O número de caracteres que você gostaria em seus pedaços. 50, 100, 100.000, etc.
- `chunk_overlap` - A quantidade que você gostaria que seus pedaços sequenciais se sobrepusessem. Isso é para tentar evitar cortar uma única parte do contexto em várias partes. Isso criará dados duplicados entre blocos.
#### Por que dividir documentos

- Para lidar mais facilmente com documentos que não seguem uma linearidade estrutural.
- Para superar o limite de contexto de entrada dos modelos de linguagem.
- Para melhorar a qualidade da representação do texto em formato de embeddings, melhorando a recuperação.
- Otimização de recursos computacionais.

#### Tipos:

#### Baseado em comprimento:
A estratégia mais intuitiva é dividir documentos com base em seu comprimento. Essa abordagem simples, porém eficaz, garante que cada pedaço não exceda um limite de tamanho especificado. Principais benefícios da divisão com base no comprimento:

- Implementação simples
- Tamanhos de pedaços consistentes
- Facilmente adaptável a diferentes requisitos de modelo

Tipos de divisão baseada em comprimento:

- **Baseado em tokens**.
- **Baseado em caracteres**.



Splitter: `RecursiveCharacterTextSplitter`
### Baseado em caractere:
O texto é naturalmente organizado em unidades hierárquicas, como parágrafos, frases e palavras. Podemos aproveitar essa estrutura inerente para informar nossa estratégia de divisão, criando uma divisão que mantém o fluxo natural da linguagem, mantém a coerência semântica dentro da divisão e se adapta a vários níveis de granularidade do texto.

A lista padrão é `separators = ["\n\n", "\n", " ", ""]`. Isso tem o efeito de tentar manter todos os parágrafos (e então frases, e então palavras) juntos o máximo possível, já que esses genericamente parecem ser os pedaços de texto semanticamente mais fortes relacionados.

Vamos analisar os parâmetros definidos para `RecursiveCharacterTextSplitter`:

- `chunk_size`: O tamanho máximo de um pedaço, onde o tamanho é determinado por `length_function`.
- `chunk_overlap`: Sobreposição de alvo entre blocos. Blocos sobrepostos ajudam a mitigar a perda de informações quando o contexto é dividido entre blocos.
- `length_function`: Função que determina o tamanho do bloco.
- `is_separator_regex`: Se a lista de separadores (padrão `["\n\n", "\n", " ", ""]`) deve ser interpretada como regex.

Splitter: `CharacterTextSplitter`
### Baseado em estrutura de documento:
Alguns documentos têm uma estrutura inerente, como arquivos HTML, Markdown (MD) ou JSON. Nesses casos, é benéfico dividir o documento com base em sua estrutura, pois ele geralmente agrupa naturalmente textos semanticamente relacionados. Principais benefícios da divisão com base na estrutura:

- Preserva a organização lógica do documento
- Mantém o contexto dentro de cada bloco
- Pode ser mais eficaz para tarefas posteriores, como recuperação ou sumarização

Por exemplo, podemos dividir um documento MD em cada nível de título, ou seja, mapeando a divisão a ser realizada pelos hastags (`#` Titulo nível 1, `##` Titulo nível 1, etc.).
Tipos:

- Markdown: [Documentação](https://python.langchain.com/docs/how_to/markdown_header_metadata_splitter/)
- HTML: [Documentação](https://python.langchain.com/docs/how_to/split_html/)
- JSON [Documentação](https://python.langchain.com/docs/how_to/recursive_json_splitter/)

#### Baseado na semântica das frases:

A divisão baseada em semântica realmente considera o _conteúdo_ do texto. Enquanto outras abordagens usam a estrutura do documento ou do texto como proxies para significado semântico, este método analisa diretamente a semântica do texto. Existem várias maneiras de implementar isso, mas conceitualmente a abordagem é dividir o texto quando há mudanças significativas no _significado_ do texto . Como exemplo, podemos usar uma abordagem de janela deslizante para gerar embeddings e comparar os embeddings para encontrar diferenças significativas:

- Comece com as primeiras frases e gere uma incorporação.
- Passe para o próximo grupo de frases e gere outra incorporação (por exemplo, usando uma abordagem de janela deslizante).
- Compare as incorporações para encontrar diferenças significativas, que indicam potenciais "pontos de ruptura" entre seções semânticas.

Essa técnica ajuda a criar blocos que são semanticamente mais coerentes, melhorando potencialmente a qualidade de tarefas posteriores, como recuperação ou sumarização.

Documentação deste divisor: https://python.langchain.com/docs/how_to/semantic-chunker/
#### Mais informações:

Para outros exemplos, consulte a documentação:
[1 - Text Splitters](https://python.langchain.com/docs/concepts/text_splitters/)
[2 - Tipos de Splitters](https://python.langchain.com/docs/how_to/#text-splitters)
[3 - Exemplo de demonstração dos chunks](https://chunkviz.up.railway.app/)

#### Prática:

Considere o seguinte texto exemplo como base:



In [ ]:
%run ../helpers/00-llm.ipynb


Vamos criar um documento dele como se fosse feito obtido via `document loaders`:

In [3]:
 
text="""A inteligência artificial (IA) é uma área da ciência da computação que tem revolucionado diversas indústrias e aspectos da vida cotidiana. Mas, o que realmente significa "inteligência artificial"? Trata-se de sistemas computacionais capazes de realizar tarefas que, anteriormente, só poderiam ser executadas por seres humanos, como reconhecimento de fala, tomada de decisão e aprendizado com dados. Impressionante, não é? Esses sistemas utilizam algoritmos avançados e grandes volumes de dados para identificar padrões, adaptarem-se a novas situações e fornecerem soluções inovadoras.

Um dos maiores avanços recentes em IA é o aprendizado profundo (ou deep learning). Essa técnica permite que máquinas realizem tarefas extremamente complexas, como diagnosticar doenças a partir de imagens médicas ou até mesmo compor músicas! Curioso como isso funciona? Redes neurais artificiais – inspiradas no funcionamento do cérebro humano – processam informações em múltiplas camadas, identificando nuances que seriam impossíveis para métodos tradicionais. Como resultado, a IA tem transformado áreas como saúde, finanças e transporte, promovendo eficiência e inovação em escala global.

No entanto, a expansão da inteligência artificial também levanta questões importantes. Estamos preparados para lidar com os desafios éticos que a IA traz? Por exemplo: como garantir que algoritmos de IA sejam imparciais e inclusivos? Além disso, há preocupações sobre o impacto no mercado de trabalho – algumas profissões podem ser substituídas por máquinas. Apesar desses desafios, uma coisa é certa: a inteligência artificial já não é mais uma tecnologia do futuro; é uma realidade do presente, moldando o mundo ao nosso redor com potencial ilimitado!"""
 

In [4]:
from langchain_core.documents import Document

from helpers.llm import initialize_llm, logger, pretty_print

llm, _, embeddings = initialize_llm()


texto_original = Document(page_content=text)

# Vamos criar agora uma lista de documentos:
docs = [texto_original]   


INFO:helpers.llm:Using AzureOpenAI.


#### Exemplo de divisão por comprimento de caracteres/tokens:

`RecursiveCharacterTextSplitter` => é um divisor recursivo, ou seja, quando ele recebe uma lista de separadores, ele vai aplicando até manter os chunks no tamanho informador pelo usuário em '`chunk_size`'. Para dividir em caracteres a lista de separadores que devemos informar é: `separators=[""]`. Dessa forma, o splitter entende que deve analisar caracteres.


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter  

text_splitter = RecursiveCharacterTextSplitter(  
    chunk_size=40,  # tamanho dos pedaços  
    chunk_overlap=5,  # sobreposição de pedaços  
    length_function=len,  # tipo de divisão: por caractere  
    separators=[""],  
)  
texts = text_splitter.split_documents(docs)  
i = 0  
for pedaco in texts:  
    print("--" * 30)  
    print(f"Chunk: {i}")  
    print("--" * 30)  
    print(pedaco)  
    print("--" * 30)  
    i += 1

------------------------------------------------------------
Chunk: 0
------------------------------------------------------------
page_content='A inteligência artificial (IA) é uma áre'
------------------------------------------------------------
------------------------------------------------------------
Chunk: 1
------------------------------------------------------------
page_content='a área da ciência da computação que tem'
------------------------------------------------------------
------------------------------------------------------------
Chunk: 2
------------------------------------------------------------
page_content='tem revolucionado diversas indústrias e'
------------------------------------------------------------
------------------------------------------------------------
Chunk: 3
------------------------------------------------------------
page_content='ias e aspectos da vida cotidiana. Mas, o'
------------------------------------------------------------
----------

Para dividir por tokens, usar a função `from_tiktoken_encoder`:

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter  

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(separators=[""], encoding_name="cl100k_base",chunk_size=100, chunk_overlap=20)  
texts = text_splitter.split_documents(docs)  
i = 0  
for pedaco in texts:  
    print("--" * 30)  
    print(f"Chunk: {i}")  
    print("--" * 30)  
    print(pedaco)  
    print("--" * 30)  
    i += 1

#### Exemplo de divisão por caracteres configuráveis:

`CharacterTextSplitter`=> é um separador que identifica o caractere indicado no parâmetro `separator` e o utiliza para separar os blocos, independente se o chunk ficar grande ou pequeno. Por exemplo, se você solicita ao splitter uma quebra por parágrafos (\n\n), ele apenas vai localizar todos os "\n\n" e criar um bloco (chunk). Tipos de divisões:

- `\n\n` - divisão por parágrafos
- `\n` - divisão por sentenças
- `" "` - divisão por palavras
- `""` - divisão por caractere

In [ ]:
from langchain_text_splitters import CharacterTextSplitter  
  
text_splitter = CharacterTextSplitter(  
    separator="\n\n", # dividir por paragrafos  
    chunk_size=1000,  
    chunk_overlap=200,  
    length_function=len,  
    is_separator_regex=False,  
)  
texts = text_splitter.split_documents(docs)  
  
i = 0  
for pedaco in texts:  
    print("--" * 30)  
    print(f"Chunk: {i}")  
    print("--" * 30)  
    print(pedaco)  
    print("--" * 30)  
    i += 1

#### Exemplo de divisão de arquivos Markdown

Primeiro vamos ler um arquivo MD e colocar numa variável python:

In [ ]:
caminho = r'../00-course-setup/SETUP.md'  

with open(caminho) as f:  
    arquivo = f.read()

Para especificar em quais cabeçalhos dividir, especifique `headers_to_split_on`ao instanciar, separador `MarkdownHeaderTextSplitter`:


In [ ]:
headers_to_split_on = [  
    ("#", "Header 1"),  
    ("##", "Header 2"),  
    ("###", "Header 3"),  
]

Agora forneça o mapeamento de divisão acima para o separador e aplique `split_text` uma vez que estamos quebrando um texto:


In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter  

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on)  
resultado_com_split_de_cabecalho = markdown_splitter.split_text(arquivo)

 

i = 0  
for content in resultado_com_split_de_cabecalho:  
    print("--" * 30)  
    print(f"Chunk: {i}")  
    print("--" * 30)  
    print(content.page_content)  
    print("--" * 30)  
    i += 1